# Dynamic model

ReviewNB link: https://app.reviewnb.com/orianebro/ArchiCrop/blob/main/dynamic_model.ipynb

In [1]:
#| echo: false

# Ignore warning about depreciated modules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
#| echo: false

# Import path of model
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, './src')

In [3]:
#| echo: false

# Enable interactive plotting
%matplotlib ipympl

## Conceptual diagram

![Conceptual diagram of shoot FSPM (not final version, some errors)](images/shoot_FSPM_conceptual_diagram_alone.jpg "Conceptual diagram")

The main stem grows according to the crop model's plant height. At the end of the growth of an internode, a leaf appears. The appearance of a leaf is triggered by fixed phyllochron (different for tillers?).  

Cf Fournier et al, 2005 for leaf elongation 

The angle of the leaf with its axis is leaf-age dependent. The leaf blade curvature depends on the angle with the axis and the leaf length.  

The reproductive organs appear from a given thermal time, when the leaves have stopped growing. 

Leaf senescence is constrained by crop model, as it has a direct influence on LAI (green LAI vs. yellow LAI ?).  

The branching pattern is set (Tillers (Zhu et al., 2015)). 

## Time-dependent parametrization of leaf area

$$ \mathcal{A_normalized} = 2 * |\int_{0}^{1} \mathcal{C}(s(u))ds(u)| $$
$$ \mathcal{A} = \mathcal{A_scaled} = 2 * w * |\int_{0}^{L} \mathcal{C}(\frac{s(u)}{L})d\frac{s(u)}{L}| $$
$$ \frac{d\mathcal{A}}{dt} = 2 * w * |\int_{ds(t)/L} \mathcal{C}(\frac{s(u)}{L})d\frac{s(u)}{L}| $$

- $\mathcal{A}$ : final leaf area (or surface), in $cm^2$ (given by crop model)
- $\frac{d\mathcal{A}}{dt}$ : gain in area, in $cm^2$, for a given leaf for a given time step $dt$ (given by crop model)  
- $\mathcal{C}$ : curve of the evolution of the leaf width along the leaf, **from tip to base** (literature/expert knowledge/observation)
- $s(t)$ : curvilinear abscissa of the midrib, in $cm$, as a function of thermal time $t$, such that: $ ds(t) = \sqrt{(dx(t))^2+(dy(t))^2} $ (computed from leaf curvature)
- $w$ : final maximal width of the leaf, in $cm$ (to fix/vary within a reasonable range)
- $L$ : final length of the leaf, in $cm$ (to fix/vary within a reasonable range)
- $t$ : thermal time, in $^oC.day$


The objective is to find, for each growing leaf, at each time step, and for a given leaf area provided to the leaf $\frac{d\mathcal{A}}{dt}$, how much length $ds(t)$ is added to the leaf.

$$ \frac{d\mathcal{A}}{dt} = 2w |\int_{s(t-1)/L}^{s(t)/L} \mathcal{C}(\frac{s(u)}{L})d\frac{s(u)}{L}| $$

According to the Fundamental Theorem of Calculus, $\mathcal{C}$ being at least $C^0$ on $[0,L]$, $F$ being the antiderivative of $\mathcal{C}$, we can write:

$$ \frac{d\mathcal{A}}{dt} = 2w (|F(\frac{s(t)}{L}) - F(\frac{s(t-1)}{L})|) $$

$$ F(\frac{s(t)}{L}) = |\frac{1}{2w} \frac{d\mathcal{A}}{dt} + F(\frac{s(t-1)}{L})| $$

Knowing the other variables, we know have to determine $s(t)$ at thermal time $t$.

In [4]:
#| echo: false
# drawing

## Apparition and growth of phytomers (stem and leaf)

The height of the plant/canopy (here we consider the height of the main stem) is given by the crop model.  
The plastochron (i.e. leaf initiation rate, in $leaf^{-1}$) is the time interval between two successive leaves produced at the SAM.  
The phyllochron (i.e. leaf emergence rate, in $leaf^{-1}$) is the time interval between two successive leaves emerging from the top of the former leaf sheath.  
The time for a leaf to grow is roughly between 1.5 and 2 phyllochrons.  
We consider a linear (rough approximation of beta function) growth for the leaf.  

- $\phi$ : phyllochron, in $^oC.day.leaf^{-1}$ (literature)
- cur_len_int : current length of growing internodes
- fin_len_int : final length of internodes, in $cm$ (literature, bell shaped model too ?)
- height : plant height, in $cm$ (given by crop model)
- $[s(t)]$ : current length of growing leaves, in $cm$ (computed before)
- fin_len_leaf : final length of leaves, in $cm$  (bell shaped model)
- leaf expansion duration, as $x * \phi$, in $^oC.day$ (literature, cf Clerget et al., 2008, etc)  

Cf Fournier et al, 2005; Stewart and Dwyer, 1993; Skinner and Nelson, 1995  



From crop model: 

## Modifying the MTG of a growing plant

There are several ways to conceive the MTG for a growing plant.  

At each thermal time step, the turtle visits each element and adds a value to a time series for each (almost) properties of the MTG.   

**OR**   

At each thermal time step, the MTG is replicated and the growing elements ar modified, and new elements are added if needed.  

In [5]:
## Imports

# from installed packages
import numpy as np
import matplotlib.pyplot as plt
from openalea.plantgl.all import Vector3
from oawidgets.plantgl import *

# from archicrop
from archicrop.cereals_leaf import parametric_leaf
from archicrop.plant_shape import geometric_dist, bell_shaped_dist
from archicrop.plant_design import leaf_azimuth
from archicrop.cereals import build_shoot
from archicrop.display import display_mtg, build_scene, display_scene

# Enable plotting with PlantGL
%gui qt

# Set nice color for plants
nice_green=Color3((50,100,0))

## Code for generating a 3D cereal shoot from descritive parameters
# Parameters 
height=1000                 # from crop model
nb_phy=17                   # fixed max nb of phytomers
max_leaf_length=70 
insertion_angle=40
scurv=0.7
curvature=80
phyllotactic_angle=180
spiral=False

diam_base=2.5
diam_top=1.0
# Lejeune and Bernier formula + col =
nb_young_phy = int(round((nb_phy - 1.95) / 1.84 / 1.3))
# stem diameters
diameter = ([diam_base] * nb_young_phy + np.linspace(diam_base, diam_top, nb_phy - nb_young_phy).tolist()) 

# Functions calls
insertion_heights=np.array(geometric_dist(height, 
                                          nb_phy, 
                                          q=1.25)) # further separate stem and pseudo stem, cf simple maize

leaf_lengths=np.array(bell_shaped_dist(max_leaf_length=max_leaf_length, 
                                       nb_phy=nb_phy, 
                                       rmax=0.7, 
                                       skew=0.15)) # plant area --> max leaf length
# leaf_areas=bell_shaped_dist(plant_area=1, nb_phy=15, rmax=0.7, skew=0.15) # cf blade_dimension

a_leaf = parametric_leaf(nb_segment=10, 
                         insertion_angle=insertion_angle, 
                         scurv=scurv, 
                         curvature=curvature, 
                         alpha=-2.3)

leaf_shapes = [a_leaf for l in leaf_lengths] # possible to replace leaf_length by nb_phy or...

leaf_azimuths = leaf_azimuth(size=len(leaf_lengths), 
                             phyllotactic_angle=phyllotactic_angle, 
                             phyllotactic_deviation=0, 
                             plant_orientation=0, 
                             spiral=spiral)

shoot, g = build_shoot(stem_diameters=diameter, 
                              insertion_heights=insertion_heights, 
                              leaf_lengths=leaf_lengths, 
                              leaf_areas=None, 
                              leaf_shapes=leaf_shapes, 
                              leaf_azimuths=leaf_azimuths)

# Build and display scene
scene, nump = build_scene(g, 
                          leaf_material=Material(nice_green), 
                          stem_material=Material(nice_green))
# display_scene(scene_single) # display in separate window
PlantGL(scene) # display in notebook

    plant  ntop    L_blade   W_blade     S_blade  form_factor       h_ins  \
0       1    17  24.257403  2.425740   44.131620         0.75    4.586218   
1       1    16  28.117999  2.811800   59.296642         0.75    5.732772   
2       1    15  32.484958  3.248496   79.145439         0.75    7.165965   
3       1    14  37.319154  3.731915  104.453946         0.75    8.957456   
4       1    13  42.533063  4.253306  135.679608         0.75   11.196820   
5       1    12  47.980052  4.798005  172.656402         0.75   13.996025   
6       1    11  53.447544  5.344754  214.247996         0.75   17.495032   
7       1    10  58.657235  5.865723  258.050339         0.75   21.868790   
8       1     9  63.275562  6.327556  300.284754         0.75   27.335987   
9       1     8  66.936782  6.693678  336.039957         0.75   34.169984   
10      1     7  69.279037  6.927904  359.968873         0.75   42.712480   
11      1     6  69.990783  6.999078  367.403232         0.75   53.390600   

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [6]:
from archicrop.plant_shape import compute_leaf_area

compute_leaf_area(g)

(0, [])

In [7]:
from ipywidgets import interact, interactive, IntSlider, fixed
from archicrop.dynamic import grow_plant, grow_plant_and_display

def grow_plant_and_display_in_NB(g, time):
    g, scene, nump=grow_plant_and_display(g, time)
    w=PlantGL(scene, group_by_color=True)
    w.wireframe=True
    return w

In [8]:
# max_time = max(g.property('end_tt').values())
interact(grow_plant_and_display_in_NB, g=fixed(g), time=IntSlider(min=20, max=2000, step=100, value=1000))

interactive(children=(IntSlider(value=1000, description='time', max=2000, min=20, step=100), Output()), _dom_c…

<function __main__.grow_plant_and_display_in_NB(g, time)>

In [9]:
from oawidgets.mtg import plot as MTGw
MTGw(g, properties='label')
# g.display()

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [10]:
g.properties()

{'edge_type': {1: '/',
  2: '/',
  4: '+',
  5: '<',
  6: '+',
  7: '<',
  8: '+',
  9: '<',
  10: '+',
  11: '<',
  12: '+',
  13: '<',
  14: '+',
  15: '<',
  16: '+',
  17: '<',
  18: '+',
  19: '<',
  20: '+',
  21: '<',
  22: '+',
  23: '<',
  24: '+',
  25: '<',
  26: '+',
  27: '<',
  28: '+',
  29: '<',
  30: '+',
  31: '<',
  32: '+',
  33: '<',
  34: '+',
  35: '<',
  36: '+'},
 'label': {1: 'Plant',
  2: 'MainAxis',
  3: 'Stem',
  4: 'Leaf',
  5: 'Stem',
  6: 'Leaf',
  7: 'Stem',
  8: 'Leaf',
  9: 'Stem',
  10: 'Leaf',
  11: 'Stem',
  12: 'Leaf',
  13: 'Stem',
  14: 'Leaf',
  15: 'Stem',
  16: 'Leaf',
  17: 'Stem',
  18: 'Leaf',
  19: 'Stem',
  20: 'Leaf',
  21: 'Stem',
  22: 'Leaf',
  23: 'Stem',
  24: 'Leaf',
  25: 'Stem',
  26: 'Leaf',
  27: 'Stem',
  28: 'Leaf',
  29: 'Stem',
  30: 'Leaf',
  31: 'Stem',
  32: 'Leaf',
  33: 'Stem',
  34: 'Leaf',
  35: 'Stem',
  36: 'Leaf'},
 'mature_length': {3: 4.586217578480434,
  5: 1.1465543946201082,
  7: 1.433192993275136,
  9: 1.79

In [11]:
## Code for generating a growing cereal shoot from descritive parameters

# Initialize the list of plant MTGs at different stages
growing_plants=[]
# positions=[(x,0,0) for x in range(0, 3600, 100)]
positions=[(x,0,0) for x in range(0, 600, 100)]

# For loop for generating plants at different stages
for time in range(0,1800,50):
# for time in range(1000,1300,50):
    gg=grow_plant(g, time=time).sub_mtg(grow_plant(g, time=time).root) 
    # Fill the list with MTG of growing plant
    growing_plants.append(gg)

    print(compute_leaf_area(gg))
    

    # i=0
    # for v in g.vertices():
    #     n=gg.node(v)
    #     if n.label is not None and n.label.startswith('Stem'):
    #         print(n.label, i, n.visible_length)
    #     i+=1
    # print('')

# Build and display scene
# scene_grow, nump = build_scene(growing_plants, positions, leaf_material=Material(nice_green), stem_material=Material(nice_green))
# # display_scene(scene_grow)
# PlantGL(scene_grow)

(374.2249455435086, [0.0, 369.9636531617556, 4.261292381753028])
(386.5607141544442, [12.33576861093561, 369.9636531617556, 4.261292381753028])
(415.2932260633141, [41.06828051980549, 369.9636531617556, 4.261292381753028])
(458.9739018562928, [73.7853448407042, 10.96361147207996, 369.9636531617556, 4.261292381753028])
(494.50777067880506, [73.7853448407042, 46.49748029459225, 369.9636531617556, 4.261292381753028])
(547.2293791252436, [73.7853448407042, 90.72060237418295, 8.498486366847741, 369.9636531617556, 4.261292381753028])
(589.6669888212084, [73.7853448407042, 90.72060237418295, 50.93609606281261, 369.9636531617556, 4.261292381753028])
(653.2960627953578, [73.7853448407042, 90.72060237418295, 109.42356888758815, 5.141601149373934, 369.9636531617556, 4.261292381753028])
(701.5422359587084, [73.7853448407042, 90.72060237418295, 109.42356888758815, 53.387774312724446, 369.9636531617556, 4.261292381753028])
(778.5667285274844, [73.7853448407042, 90.72060237418295, 109.42356888758815,

## Fit plant growth and development to LAI(t) and height(t) curves

LAI(t) curve

In [12]:
def linear_LA(x, slope=5, intercept=0.0):
    """
    Generate a linear curve.

    Args:
    - x: Input values
    - slope: Slope of the line
    - intercept: Intercept of the line

    Returns:
    - y: Output values corresponding to the linear curve
    """
    y = slope * x + intercept
    return y

Two constraints :

$$ \mathcal{A} = \mathcal{A_scaled} = 2 * w * |\int_{0}^{L} \mathcal{C}(\frac{s(u)}{L})d\frac{s(u)}{L}| $$
$$ \text{leaf length} = L * e^{-\log(skew) * (2 * (r - rmax)^2 + (r - rmax)^3)} $$
r = np.linspace(1. / nb_phy, 1, nb_phy)

In [13]:
from archicrop.plant_shape import compute_leaf_area

compute_leaf_area(g)

(5876.376959353999,
 [73.7853448407042,
  90.72060237418295,
  109.42356888758815,
  201.67537532632647,
  245.077108970223,
  288.67084540115076,
  327.7876562446977,
  356.77089567182355,
  579.7774334129538,
  606.9865794382832,
  601.8457357316205,
  562.1067475553564,
  491.6970230418619,
  627.4649058939882,
  504.04024269510165,
  208.54689386813664])

In [14]:
from scipy.optimize import curve_fit

# Define your parametric model as a Python function
def parametric_model(x):
    """
    Define your parametric model here.

    Args:
    - x: Input values
    - param1, param2, ...: Parameters of your model

    Returns:
    - y: Output values corresponding to the model
    """
    for t in time
        # Compute your model's output based on input values and parameters
        ## Code for generating a 3D cereal shoot from descritive parameters
        # Parameters 
        diam_base=2.5
        diam_top=1.0
        # Lejeune and Bernier formula + col =
        nb_young_phy = int(round((nb_phy - 1.95) / 1.84 / 1.3))
        # stem diameters
        diameter = ([diam_base] * nb_young_phy + np.linspace(diam_base, diam_top, nb_phy - nb_young_phy).tolist()) 
        height=1000                 # from crop model
        nb_phy=17                   # fixed max nb of phytomers
        max_leaf_length=70 
        insertion_angle=40
        scurv=0.7
        curvature=80
        phyllotactic_angle=180
        spiral=False
        
        # Functions calls
        insertion_heights=np.array(geometric_dist(height, 
                                                  nb_phy, 
                                                  q=1.25)) # further separate stem and pseudo stem, cf simple maize
        
        leaf_lengths=np.array(bell_shaped_dist(max_leaf_length=max_leaf_length, 
                                               nb_phy=nb_phy, 
                                               rmax=0.7, 
                                               skew=0.15)) # plant area --> max leaf length
        # leaf_areas=bell_shaped_dist(plant_area=1, nb_phy=15, rmax=0.7, skew=0.15) # cf blade_dimension
        
        a_leaf = parametric_leaf(nb_segment=10, 
                                 insertion_angle=insertion_angle, 
                                 scurv=scurv, 
                                 curvature=curvature, 
                                 alpha=-2.3)
        
        leaf_shapes = [a_leaf for l in leaf_lengths] # possible to replace leaf_length by nb_phy or...
        
        leaf_azimuths = leaf_azimuth(size=len(leaf_lengths), 
                                     phyllotactic_angle=phyllotactic_angle, 
                                     phyllotactic_deviation=0, 
                                     plant_orientation=0, 
                                     spiral=spiral)
        
        shoot, g = build_shoot(stem_diameters=stem_radius, 
                                      insertion_heights=insertion_heights, 
                                      leaf_lengths=leaf_lengths, 
                                      leaf_areas=None, 
                                      leaf_shapes=leaf_shapes, 
                                      leaf_azimuths=leaf_azimuths)
    y = compute_leaf_area(g)
    # # Build and display scene
    # scene, nump = build_scene(g, 
    #                           leaf_material=Material(nice_green), 
    #                           stem_material=Material(nice_green))
    # # display_scene(scene_single) # display in separate window
    # PlantGL(scene) # display in notebook
    return y

# Prepare your data
x_data = []
y_data = []
for t in range(0,1700,100):
    x_data.append(t)
    y_data.append(linear_LA(t))
x_data = np.array(x_data)  # Your input data
y_data = np.array(y_data)  # Your output data

# Use scipy.optimize.curve_fit to fit your model to the data and obtain optimal parameters
initial_guess = [...]  # Initial guess for your parameters
optimal_params, covariance = curve_fit(parametric_model, x_data, y_data, p0=initial_guess)

# Print the optimal parameters
print("Optimal Parameters:", optimal_params)

# Optionally, visualize the fit
import matplotlib.pyplot as plt
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, parametric_model(x_data, *optimal_params), color='red', label='Fitted Curve')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

SyntaxError: invalid syntax (1157590130.py, line 15)

## Next steps

- Bending dynamics of leaves and tillers